### Aufgaben

Hier werden für den Stichprobenumfang n=100

- zur Wahrscheinlichkeit p=0,26 das 95%-Prognoseintervall,
- zur relativen Häufigkeit h=0,18 das 95%-Konfidenzintervall
- exakt und auch näherungsweise berechnet.

a) Zeigen Sie, dass die Ergebnisse stimmen.

b) Fassen Sie die Bedeutung der gelben und der grünen Strecke in Worte.

c) Elif: "Ich kann der Abbildung die Antwort auf die Frage entnehmen, ob die relative Häufigkeit h=0,18 Anlass bietet, die Wahrscheinlichkeit p=0,26 zu bezweifeln." Erläutern Sie Elifs Aussage und zu welchem Ergebnis Elif kommt.

d) Ümü: "Die Antwort auf Elifs Frage hängt davon ab, ob man exakt oder mit der Näherung arbeitet." Prüfen und kommentieren Sie.

In [187]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import norm, beta
from scipy.optimize import root_scalar
import warnings

warnings.simplefilter("ignore")

def wilson_ci_bounds(h, n, c):
    A = 1 + c**2 / n
    B = -2 * h - c**2 / n
    C = h**2
    
    def equation(p):
        return A * p**2 + B * p + C
    
    lower_bound = root_scalar(equation, bracket=[0, h], method='brentq').root
    upper_bound = root_scalar(equation, bracket=[h, 1], method='brentq').root
    
    return lower_bound, upper_bound

def clopper_pearson_ci(h, n,c):
    alpha = 2 * (1 - norm.cdf(c))  # Berechnung von alpha aus c
    lower_bound = beta.ppf(alpha / 2, h, n - h + 1)
    upper_bound = beta.ppf(1 - alpha / 2, h + 1, n - h)
    return lower_bound, upper_bound

def wald_ci(h, n, c):
    se = np.sqrt(h * (1 - h) / n)
    lower_bound = h - c * se
    upper_bound = h + c * se
    return max(0, lower_bound), min(1, upper_bound)

def plot_confidence_ellipse(h, n, c, p_value):
    p = np.linspace(0, 1, 1000)  # p-Werte als x-Achse
    f1 = p - c * np.sqrt(p * (1 - p) / n)
    f2 = p + c * np.sqrt(p * (1 - p) / n)
    lower_wilson, upper_wilson = wilson_ci_bounds(h, n, c)
    lower_cp, upper_cp = clopper_pearson_ci(h * n, n,c)
    lower_wald, upper_wald = wald_ci(h, n, c)
    
    # Prognoseintervall für gegebenes p
    f1_p = p_value - c * np.sqrt(p_value * (1 - p_value) / n)
    f2_p = p_value + c * np.sqrt(p_value * (1 - p_value) / n)
    
    plt.figure(figsize=(6,6))
    plt.plot(p, f1, 'r', label="$f_-(p)$")
    plt.plot(p, f2, 'b', label="$f_+(p)$")
    plt.axhline(h, color='gray', linestyle='dashed')
    plt.axvline(p_value, color='gray', linestyle='dashed')
    
    # Prognoseintervall
    plt.plot([p_value, p_value], [f1_p, f2_p], 'm-', linewidth=3, label="Prognoseintervall")
    
    # Wilson-Konfidenzintervall (grün)
    plt.plot([lower_wilson, upper_wilson], [h, h], 'g-', linewidth=3, label="Wilson-KI")
    
    # Clopper-Pearson-Konfidenzintervall (orange)
    #plt.plot([lower_cp, upper_cp], [h, h], 'orange', linewidth=2, label="Clopper-Pearson-KI")
    
    # Wald-Konfidenzintervall (blau)
    #plt.plot([lower_wald, upper_wald], [h, h], 'b-', linewidth=2, label="Wald-KI")
    
    plt.scatter([lower_wilson, upper_wilson], [h, h], color='black', s=20, zorder=3)
    plt.scatter([p_value, p_value], [f1_p, f2_p], color='black', s=20, zorder=3)
    #plt.scatter([lower_cp, upper_cp], [h, h], color='red', zorder=3)
    #plt.scatter([lower_wald, upper_wald], [h, h], color='blue', zorder=3)
    plt.fill_between(p, f1, f2, color='gray', alpha=0.1)
    
    # Punkte (0,h) und (p,0)
    plt.scatter([0, p_value], [h, 0], color='black', marker='o', zorder=3)
    
    plt.xlabel("p (unbekannter Parameter)",fontsize=12)
    plt.ylabel("h (Stichprobenergebnis)",fontsize=12)
    plt.legend(fontsize=11)
    plt.title(f"Wilson-Konfidenzellipse ({round(norm.cdf(c) * 2 - 1, 3) * 100}% Konfidenzintervall)",
              fontsize=12)
    plt.grid(True)
    plt.show()
    
    print(f"Prognoseintervall für p={p_value:.2f}: [{f1_p:.4f}, {f2_p:.4f}]")
    print()
    print(f"Wilson-Konfidenzintervall: [{lower_wilson:.4f}, {upper_wilson:.4f}]")
    print(f"Clopper-Pearson-Konfidenzintervall: [{lower_cp:.4f}, {upper_cp:.4f}]")
    print(f"Wald-Konfidenzintervall: [{lower_wald:.4f}, {upper_wald:.4f}]")
   

def interactive_plot(c, n, h, p_value):
    plot_confidence_ellipse(h, n, c, p_value)

c_slider = widgets.FloatSlider(min=1.0, max=3.0, step=0.01, value=1.96, description="c")
n_slider = widgets.IntSlider(min=10, max=1000, step=10, value=100, description="n")
h_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.18, description="h")
p_slider = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.26, description="p")

widgets.interactive(interactive_plot, c=c_slider, n=n_slider, h=h_slider, p_value=p_slider)


interactive(children=(FloatSlider(value=1.96, description='c', max=3.0, min=1.0, step=0.01), IntSlider(value=1…